In [1]:
from datasets import load_dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['WORLD_SIZE'] = '1'
import torch
import transformers
import ecco
from ecco import LM
from transformers import AutoTokenizer
import torch
from math import ceil
from random import choices, randint
import re
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from scipy.optimize import linear_sum_assignment
from scipy.stats import mode
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained('ai-forever/rugpt3small_based_on_gpt2')
model = transformers.AutoModelForCausalLM.from_pretrained('ai-forever/rugpt3small_based_on_gpt2')

In [4]:
config = {"embedding": "transformer.wte.weight", # String. Directly accessed
    "type": 'causal',
    "activations": ['mlp\.c_proj'],
    "token_prefix": 'Ġ',
    "partial_token_prefix": '',
    "tokenizer_config": {"token_prefix": 'Ġ',"partial_token_prefix": ''}}

In [5]:
lm = ecco.lm.LM(model=model, tokenizer=tokenizer, model_name='rugpt', config=config,collect_activations_flag=True)

In [27]:
s = tokenizer.batch_encode_plus(['Мама мыла раму.', 'Вася мыл красную машину.'], return_tensors='pt', padding='longest', truncation=True)

In [28]:
s

{'input_ids': tensor([[    0, 35695, 43068,   351,   597,    18],
        [37265,   656,   271, 35969,  5894,    18]]), 'attention_mask': tensor([[0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [29]:
a = lm(s)

In [209]:
b.activations['decoder'].shape

(16, 12, 3072, 128)

In [212]:
acts = b.activations['decoder']
loc_acts = np.zeros(acts.shape[:-1])
for i, t in enumerate(loc):
    loc_acts[i] = acts[i,:,:,t]

In [219]:
loc_acts.shape

(16, 12, 3072)

In [17]:
list(map(lambda x: x[0],list(model.named_parameters())))

['transformer.wte.weight',
 'transformer.wpe.weight',
 'transformer.h.0.ln_1.weight',
 'transformer.h.0.ln_1.bias',
 'transformer.h.0.attn.c_attn.weight',
 'transformer.h.0.attn.c_attn.bias',
 'transformer.h.0.attn.c_proj.weight',
 'transformer.h.0.attn.c_proj.bias',
 'transformer.h.0.ln_2.weight',
 'transformer.h.0.ln_2.bias',
 'transformer.h.0.mlp.c_fc.weight',
 'transformer.h.0.mlp.c_fc.bias',
 'transformer.h.0.mlp.c_proj.weight',
 'transformer.h.0.mlp.c_proj.bias',
 'transformer.h.1.ln_1.weight',
 'transformer.h.1.ln_1.bias',
 'transformer.h.1.attn.c_attn.weight',
 'transformer.h.1.attn.c_attn.bias',
 'transformer.h.1.attn.c_proj.weight',
 'transformer.h.1.attn.c_proj.bias',
 'transformer.h.1.ln_2.weight',
 'transformer.h.1.ln_2.bias',
 'transformer.h.1.mlp.c_fc.weight',
 'transformer.h.1.mlp.c_fc.bias',
 'transformer.h.1.mlp.c_proj.weight',
 'transformer.h.1.mlp.c_proj.bias',
 'transformer.h.2.ln_1.weight',
 'transformer.h.2.ln_1.bias',
 'transformer.h.2.attn.c_attn.weight',
 'tra

In [45]:
ds = load_dataset('cointegrated/taiga_stripped_proza')

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/68 [00:00<?, ?it/s]

In [35]:
ds['train'][0]['text']

'Глава первая\n\n----- . . . -----\n\nМихаил Иванович Нарывайло практически никак не среагировал на звонок телефона, указанного в рекламе детективного бюро "НикитаточкаRU". Он безучастно смотрел на не перестававший трезвонить аппарат, словно предчувствуя, что ничего хорошего, подняв трубку, не услышит. Никита, удобно устроившись на диване, пытался, как обычно, восполнить ночной недосып. Но после нескольких звонков не выдержал и с любопытством взглянул на сидевшего за столом начальника, пытаясь угадать, возьмет Нарывайло трубку сам или поручит ему это сделать.\n-- Нам звонят, шеф, -- осторожно, чтобы ненароком не навлечь на себя начальственное недовольство, негромко сказал Никита.\n-- Не слепой, слышу, -- проворчал Михаил Иванович и, выдержав продолжительную паузу в надежде, что звонки вдруг прекратятся, нехотя взял трубку. Представившись, он через мгновение перебил звонившего фразой "Одну минуту!". Затем, прикрыв микрофон рукой, произнес:\n-- Я как знал, что ничего хорошего грядущий де

In [39]:
freq = pd.read_csv('/home/raskind/venv/RMT_interpretation/t5-experiments/analysis_memory/cpts/trigrams.csv', sep=',',names=['rank', 'text', 'docs', 'freq'])

In [40]:
pip install razdel

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [41]:
from string import punctuation
from pymorphy2 import MorphAnalyzer
from razdel import sentenize

In [42]:
m = MorphAnalyzer()

In [45]:
ana = '|'.join(list(map(lambda x: x.word,m.parse(freq['lex'][0])[0].lexeme)))

In [60]:
samples = []
neg_samples = []

In [59]:
len(samples), len(neg_samples)

(6178, 6401)

In [61]:
freq = freq[freq['text'] != 'и т д']
freq = freq[freq['text'] != 'и т п']

In [62]:
shift = 0
ind = 0
for i, word in enumerate(freq['text']):
    # if i < 171:
    #     continue
    # if word == 'и т д':
    #     continue
    print(i, word)
    ana = r',?\s*'.join(word.split())
    neg_ana = word.split()[-1]
    if not ana[0].isalpha():
        shift += 1
        continue
    while len(samples) < (i+1-shift) * 256 or len(neg_samples) < (i+1-shift) * 256:
        t = ds['train'][ind]['text']
        txts = list(sentenize(t))
        for s in txts:
            a = re.search(str(r'\b('+ana+r')\b'), s.text, re.IGNORECASE)
            neg_a = re.search(str(r'\b('+neg_ana+r')\b'), s.text, re.IGNORECASE)
            if a and len(samples) <= (i+1-shift) * 256:
                samples.append([s.text, a.span()])
            elif neg_a and len(neg_samples) <= (i+1-shift) * 256:
                neg_samples.append([s.text, neg_a.span()])
            if len(samples) >= (i+1-shift) * 256 and len(neg_samples) >= (i+1-shift) * 256:
                break
        ind = (ind+1) % len(ds['train'])
    

0 о том что
1 в том что
2 до сих пор
3 для того чтобы
4 в том числе
5 в то время
6 в это время
7 в то же
8 по крайней мере
9 то же время
10 в самом деле
11 на то что
12 к тому же
13 тем не менее
14 в конце концов
15 на самом деле
16 в связи с
17 во всяком случае
18 не может быть
19 так же как
20 за то что
21 так и не
22 с тех пор
23 на этот раз
24 в соответствии с
25 вместе с тем
26 с другой стороны
27 после того как
28 о том как
29 в настоящее время
30 с одной стороны
31 я не могу
32 бы то ни
33 тем более что
34 по отношению к
35 у нас в
36 я не знаю
37 в свою очередь
38 может быть и
39 в отличие от
40 на другой день
41 дело в том
42 чуть ли не
43 том числе и
44 в первый раз
45 в первую очередь
46 то же самое
47 в свое время
48 по сравнению с
49 в этом случае
50 что он не
51 с точки зрения
52 не только в
53 а может быть
54 не то что
55 и тут же
56 в зависимости от
57 то что он
58 что я не
59 несмотря на то
60 до тех пор
61 же как и
62 и в то
63 а между тем
64 то ни было
65 к тому что


In [182]:
anas = set()
doubles = []
shift = 0
for i, word in enumerate(freq['lex']):
    # if i < 171:
    #     continue
    # print(i, word)
    if word == 'мочь':
        ana = '|'.join(list(map(lambda x: x.word,m.parse(word)[2].lexeme)))
    else:
        ana = '|'.join(list(map(lambda x: x.word,m.parse(word)[0].lexeme)))
    if not ana[0].isalpha():
        shift += 1
        continue
    if ana in anas:
        doubles.append(i-shift)
    else:
        anas |= set([ana])

0 и
1 в
2 не
3 на
4 я
5 он
6 быть
7 что
8 с
9 а
10 как
11 то
12 она
13 к
14 этот
15 это
16 по
17 но
18 они
19 мы
20 свой
21 который
22 из
23 весь
24 у
25 за
26 от
27 все
28 о
29 так
30 же
31 вы
32 для
33 мочь
34 ты
35 год
36 один
37 его
38 тот
39 человек
40 только
41 такой
42 бы
43 себя
44 сказать
45 еще
46 мой
47 или
48 говорить
49 до
50 время
51 уже
52 когда
53 другой
54 наш
55 да
56 если
57 знать
58 вот
59 сам
60 ни
61 день
62 дело
63 при
64 стать
65 чтобы
66 самый
67 жизнь
68 очень
69 нет
70 во
71 даже
72 два
73 рука
74 ее
75 первый
76 ли
77 под
78 со
79 кто
80 где
81 новый
82 слово
83 какой
84 раз
85 теперь
86 их
87 идти
88 без
89 после
90 иметь
91 там
92 ничто
93 должен
94 большой
95 видеть
96 место
97 хотеть
98 можно
99 глаз
100 лицо
101 потом
102 ну
103 много
104 более
105 здесь
106 работа
107 дом
108 надо
109 русский
110 каждый
111 тут
112 друг
113 несколько
114 тогда
115 думать
116 сторона
117 голова
118 потому
119 жить
120 сделать
121 через
122 делать
123 стоять
124 час
125 

In [183]:
doubles = [-1] + doubles + [len(samples) // 512+1]

In [185]:
len(cleaned_samples) /  512

481.0

In [184]:
cleaned_samples = []
for d in range(1,len(doubles)):
    cleaned_samples.extend(samples[(doubles[d-1]+1) * 512:doubles[d] * 512])

In [30]:
neg_samples

[['-----\n\nМихаил Иванович Нарывайло практически никак не среагировал на звонок телефона, указанного в рекламе детективного бюро "НикитаточкаRU".',
  (51, 53)],
 ['Он безучастно смотрел на не перестававший трезвонить аппарат, словно предчувствуя, что ничего хорошего, подняв трубку, не услышит.',
  (25, 27)],
 ['Но после нескольких звонков не выдержал и с любопытством взглянул на сидевшего за столом начальника, пытаясь угадать, возьмет Нарывайло трубку сам или поручит ему это сделать.',
  (28, 30)],
 ['-- Нам звонят, шеф, -- осторожно, чтобы ненароком не навлечь на себя начальственное недовольство, негромко сказал Никита.',
  (50, 52)],
 ['-- Не слепой, слышу, -- проворчал Михаил Иванович и, выдержав продолжительную паузу в надежде, что звонки вдруг прекратятся, нехотя взял трубку.',
  (3, 5)],
 ['Затем, прикрыв микрофон рукой, произнес:\n-- Я как знал, что ничего хорошего грядущий день нам не принесет -- женщина на проводе.',
  (94, 96)],
 ['-- Долго разговаривать он не стал и предлож

In [63]:
bigs = pd.DataFrame(samples)
n_bigs = pd.DataFrame(neg_samples)

In [64]:
bigs.to_csv('/home/raskind/venv/RMT_interpretation/t5-experiments/analysis_memory/cpts/bigram_samples.csv')
n_bigs.to_csv('/home/raskind/venv/RMT_interpretation/t5-experiments/analysis_memory/cpts/neg_bigram_samples.csv')

In [71]:
bigs = pd.read_csv('/home/raskind/venv/RMT_interpretation/t5-experiments/analysis_memory/cpts/bigram_samples.csv')
n_bigs = pd.read_csv('/home/raskind/venv/RMT_interpretation/t5-experiments/analysis_memory/cpts/neg_bigram_samples.csv')

In [72]:
bigs = bigs.drop(columns=['Unnamed: 0']).rename(columns={'0':'text', '1':'loc'})
n_bigs = n_bigs.drop(columns=['Unnamed: 0']).rename(columns={'0':'text', '1':'loc'})

In [8]:
cl_s

,text,loc
0,Но после нескольких звонков не выдержал и с лю...,"(40, 41)"
1,"-- Не слепой, слышу, -- проворчал Михаил Ивано...","(50, 51)"
2,Я сейчас дам трубочку нашему ведущему специали...,"(76, 77)"
3,-- Долго разговаривать он не стал и предложил ...,"(34, 35)"
4,"Людмила Чулкова, как представилась посетительн...","(139, 140)"
...,...,...
246267,Примерно в это же время появился Александр Мал...,"(24, 32)"
246268,"Например, прыжки в высоту, где раньше блистали...","(219, 227)"
246269,Не хватал мне еще у себя на похоронах появиться.,"(38, 47)"
246270,Несмотря на все сильнее терзающие меня холод и...,"(102, 111)"


In [8]:
from torch.utils.data import Dataset, DataLoader

In [65]:
class ProbeDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, ind):
        return({'text': self.samples['text'][ind],
                'location': tuple(map(int, self.samples['loc'][ind].strip('()').split(', ')))})

In [66]:
encode_plus_kwargs = {'truncation':True,
                      'padding':'max_length',
                      'max_length': 128,
                      'return_offsets_mapping': True,
                      'return_tensors':'pt'}


def collate_fn(batch):
    # print(batch)
    texts = [b['text'] for b in batch]
    locations = torch.tensor([list(b['location']) for b in batch])
    collated = tokenizer.batch_encode_plus(texts, **encode_plus_kwargs)
    collated['location'] = torch.argwhere(collated['offset_mapping'][:,:,1] == locations[:,1:].expand(collated['offset_mapping'].shape[0], collated['offset_mapping'].shape[1]))
    if collated['location'].shape[0] > 32:
        rep = torch.cat([torch.tensor([False]),collated['location'][:-1,0] == collated['location'][1:,0]])
        collated['location'] = collated['location'][rep]
    collated.pop('offset_mapping')
    return collated


In [67]:
BS=32

In [73]:
big_ds = ProbeDataset(bigs)
big_dl = DataLoader(big_ds, batch_size = BS, collate_fn=collate_fn)
neg_ds = ProbeDataset(n_bigs)
neg_dl = DataLoader(neg_ds, batch_size = BS, collate_fn=collate_fn)

In [74]:
from tqdm import tqdm

In [75]:
active_neurons = []
cur_w_acts = []
for i, batch in tqdm(enumerate(big_dl)):
    loc = batch.pop('location')
    b = lm(batch)
    acts = b.activations['decoder']
    loc_acts = np.zeros(acts.shape[:-1])
    for s in loc:
        loc_acts[s[0]] = acts[s[0],:,:,s[1]]
    cur_w_acts.append(loc_acts)
    if not ((i+1) * BS % 256):
        active_neurons.append(np.all(np.concatenate(cur_w_acts, axis=0) > 0, axis=0).astype(int))
        # break
        cur_w_acts = []

neg_active_neurons = []
cur_w_acts = []
for i, batch in tqdm(enumerate(neg_dl)):
    loc = batch.pop('location')
    b = lm(batch)
    acts = b.activations['decoder']
    loc_acts = np.zeros(acts.shape[:-1])
    for s in loc:
        loc_acts[s[0]] = acts[s[0],:,:,s[1]]
    cur_w_acts.append(loc_acts)
    if not ((i+1) * BS % 256):
        neg_active_neurons.append(np.all(np.concatenate(cur_w_acts, axis=0) > 0, axis=0).astype(int))
        # break
        cur_w_acts = []
     

784it [10:24,  1.26it/s]
785it [10:24,  1.26it/s]


In [76]:
active_neurons.__len__()

98

In [88]:
act_n = np.stack(active_neurons).reshape(98, -1)


In [89]:
neg_n = np.stack(neg_active_neurons).reshape(98, -1)

In [78]:
np.argwhere(((act_n - neg_n) > 0).astype(int).sum(axis=1))

array([[ 0],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 9],
       [14],
       [18],
       [21],
       [22],
       [26],
       [31],
       [38],
       [39],
       [46],
       [50],
       [51],
       [60],
       [65],
       [70],
       [78],
       [89],
       [92],
       [93],
       [95],
       [99]])

In [85]:
freq

,rank,text,docs,freq
0,1,о том что,11793,37235
1,2,в том что,12321,36961
2,3,до сих пор,8947,24284
4,5,для того чтобы,7661,19722
5,6,в том числе,9206,19309
...,...,...,...,...
95,96,в данном случае,2801,4909
96,97,в этом году,3257,4864
97,98,то что я,2044,4854
98,99,и так далее,2305,4789


In [87]:
np.argwhere(((act_n - neg_n) > 0).astype(int).sum(axis=1)).flatten()

array([ 0,  3,  4,  5,  6,  9, 14, 18, 21, 22, 26, 31, 38, 39, 46, 50, 51,
       60, 65, 70, 78, 89, 92, 93, 95, 99])

In [90]:
small = freq.iloc[np.argwhere(((act_n - neg_n) > 0).astype(int).sum(axis=1)).flatten()]

In [92]:
small

,rank,text,docs,freq
2,3,до сих пор,8947,24284
4,5,для того чтобы,7661,19722
9,10,по крайней мере,5195,15865
13,14,к тому же,6187,13787
22,23,так и не,4646,10907
24,25,на этот раз,3730,10372
30,31,о том как,4387,9716
35,36,тем более что,3992,8579
37,38,у нас в,3892,8332
41,42,в отличие от,4502,7887


In [93]:
a = ((act_n - neg_n) > 0).astype(int)

In [95]:
final_df = pd.DataFrame({'word':small.iloc[list(np.argwhere(a[np.argwhere(a.sum(axis=1)> 0)])[:,0])]['text'],
'rank': small.iloc[list(np.argwhere(a[np.argwhere(a.sum(axis=1)> 0)])[:,0])]['rank'],
'layer': list(np.argwhere(a[np.argwhere(a.sum(axis=1)> 0)])[:,2] // 3072)})

In [96]:
final_df

,word,rank,layer
2,до сих пор,3,0
2,до сих пор,3,0
2,до сих пор,3,0
2,до сих пор,3,0
2,до сих пор,3,0
...,...,...,...
97,то что я,98,11
97,то что я,98,11
97,то что я,98,11
97,то что я,98,11


In [97]:
final_df.to_csv('trigr_layers.csv')

In [153]:
torch.argwhere(a['offset_mapping'][:,:,1] == a['location'][:,1:].expand(8,128))

tensor([[  0, 101],
        [  1, 109],
        [  2, 121],
        [  3, 118],
        [  4,  47],
        [  5, 109],
        [  6, 111],
        [  7,  91]])

In [160]:
a['location']

tensor([101, 109, 121, 118,  47, 109, 111,  91])

In [144]:
probe_ds[0]['text'][2:8]

' после'

In [98]:
ana = '|'.join(list(map(lambda x: x.word,m.parse('в')[0].lexeme)))

In [99]:
ana

'в|во'

In [100]:
re.search(str(r'\b('+ana+r')\b'), s.text, re.IGNORECASE)

<re.Match object; span=(32, 33), match='в'>

In [66]:
list(sentenize(ds['train'][1]['text']))[0].text

'Спустя два часа после происшествия в ее номере Кейт сидела в баре неподалеку от отеля и крутила в руках полупустой бокал с виски.'

In [34]:
ds['train'][1]['text'].lower()

'спустя два часа после происшествия в ее номере кейт сидела в баре неподалеку от отеля и крутила в руках полупустой бокал с виски. конечно, она могла бы отыскать бар на территории отеля, но сейчас это было последнее место, где она хотела бы находиться. ее всю трясло, и даже алкоголь почти не помогал. полицейские, прибывшие на место трагедии, долго задавали ей вопросы о случившемся. она рассказала все: как видела тома в аэропорту, как столкнулась с ним в холле на своем этаже. ее спрашивали о том, не встречала ли она его раньше, не было ли у него причин желать ей зла. выполнив свою работу, стражи порядка опечатали ее номер на время дальнейшего расследования, а коронеры забрали тело, лежавшее внизу, для вскрытия. ей разрешили забрать свои вещи, а администрация отеля пообещала предоставить ей другой номер. не обошлось и без вездесущих журналистов. они прибыли на место сразу же после сотрудников полицейского департамента и активно пытались пролезть на место происшествия, тем более что главн

In [9]:
freq['lex'][0]

'и'

In [96]:
i = 1006

In [97]:
samples[i][0][samples[i][1][0]-1:samples[i][1][1]+1]

' вд'

In [93]:
samples[1005]

['Или в стае.', (4, 5)]